In [1]:
import sqlite3
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from classifier import classify_text

c:\Users\Даниил\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Модель успешно загружена.
Токенизатор успешно загружен.
Кодировщик меток успешно загружен.


In [ ]:
def load_data():
    conn = sqlite3.connect(r"C:\Study\Python\WebsiteClassification\bot\data.db")
    df = pd.read_sql_query("SELECT * FROM articles", conn)
    conn.close()
    return df

In [ ]:
data = load_data()

In [4]:
data.shape

(486, 8)

In [5]:
print(data['topic'].value_counts())

topic
Искусственный интеллект и машинное обучение    57
Робототехника                                  56
Компьютеры и общество                          52
Компьютерное зрение и распознавание образов    52
Криптография и безопасность информации         48
Обработка естественного языка                  47
Человеко-компьютерное взаимодействие           45
Информационный поиск                           45
Фундаментальные основы CS                      42
Распределённые и параллельные вычисления       42
Name: count, dtype: int64


In [6]:
data['topic'] = data['topic'].str.replace('Криптография и безопасность информации', 'Криптография и безопасность')

In [7]:
print(data['topic'].value_counts())

topic
Искусственный интеллект и машинное обучение    57
Робототехника                                  56
Компьютеры и общество                          52
Компьютерное зрение и распознавание образов    52
Криптография и безопасность                    48
Обработка естественного языка                  47
Человеко-компьютерное взаимодействие           45
Информационный поиск                           45
Фундаментальные основы CS                      42
Распределённые и параллельные вычисления       42
Name: count, dtype: int64


In [ ]:
import re
def is_bad_example(row):
    combined_text = ' '.join([str(row[col]) for col in ['title', 'description', 'summary'] if pd.notna(row[col])])

    if re.search(r'[ÐÑ]', combined_text):
        return True
    
    if len(combined_text.split()) < 10:
        return True
    
    return False

filtered_data = data[~data.apply(is_bad_example, axis=1)].reset_index(drop=True)
print(f"Число записей до фильтрации: {len(data)}")
print(f"Число записей после фильтрации: {len(filtered_data)}")

Число записей до фильтрации: 486
Число записей после фильтрации: 485


In [9]:
import re
import pandas as pd
import logging

def clean_text(text):
    if not isinstance(text, str) or text == 'Нет данных':
        return

    text = text.replace('\xa0', ' ').replace('\ufeff', '')
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text or "Нет данных"

def clean_dataframe(dataframe, text_columns=None):
    if text_columns is None:
        text_columns = ['title', 'description', 'summary']
        
    cleaned_df = dataframe.copy()

    for col in text_columns:
        if col in cleaned_df.columns:
            cleaned_df[col] = cleaned_df[col].apply(clean_text)
        else:
            print(f"Столбец {col} не найден в датафрейме")
    cleaned_df = cleaned_df[cleaned_df[['title', 'summary']].apply(lambda x: x['title'] != 'Нет данных' and x['summary'] != 'Нет данных', axis=1)]
    
    logging.info(f"Очищено {len(dataframe) - len(cleaned_df)} строк с некорректным текстом")
    return cleaned_df

In [10]:
filtered_data.shape

(485, 8)

In [11]:
cleaned_df = clean_dataframe(filtered_data, text_columns=['title', 'summary'])

In [12]:
X_title = cleaned_df['title'].tolist()
X_summary = cleaned_df['summary'].tolist()
y_true = cleaned_df['topic'].tolist()

In [13]:
y_pred = []

for idx, (title, summary) in enumerate(zip(X_title, X_summary)):
    try:
        prediction = classify_text(title, summary)
        y_pred.append(prediction)
        print(f"[{idx+1}/{len(X_title)}] Предсказано: {prediction}")
    except Exception as e:
        print(f"[{idx+1}] Ошибка: {e}")
        y_pred.append("Ошибка")

Исходный текст: Компьютерное зрение — Википедия Компьютерное зрение — теория и технология создания искусственных систем, которые могут производить обнаружение, отслеживание, классификацию объектов или окружающей среды. Современное положение компьютерного зрения может быть охарактеризовано как дополнение к биологическому зрению.
Предобработанный текст: компьютерный зрение википедия компьютерный зрение теория технология создание искусственный система который мочь производить обнаружение отслеживание классификация объект окружающий среда современный положение компьютерный зрение охарактеризовать дополнение биологический зрение
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[1/485] Предсказано: Искусственный интеллект и машинное обучение
Исходный текст: Что такое компьютерное зрение и где его применяют | РБК Тренды Технологии машинного обучения позволяют научить компьютер «видеть» и извлекать информацию из увиденного. Компьютерное зрение — это область искусственного интеллекта, связанная с анализом и

In [14]:
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc:.2%}")

print("\nОтчет по классам:")
print(classification_report(y_true, y_pred))

print("\nМатрица ошибок:")
print(confusion_matrix(y_true, y_pred))

Accuracy: 70.52%

Отчет по классам:
                                             precision    recall  f1-score   support

                       Информационный поиск       0.94      0.67      0.78        45
Искусственный интеллект и машинное обучение       0.52      1.00      0.68        57
Компьютерное зрение и распознавание образов       0.91      0.41      0.57        51
                      Компьютеры и общество       0.46      0.48      0.47        52
                Криптография и безопасность       0.70      1.00      0.82        48
              Обработка естественного языка       0.91      0.91      0.91        47
   Распределённые и параллельные вычисления       0.79      0.81      0.80        42
                              Робототехника       0.87      0.73      0.80        56
                  Фундаментальные основы CS       0.64      0.43      0.51        42
       Человеко-компьютерное взаимодействие       0.78      0.56      0.65        45

                           

In [ ]:
import pandas as pd
results_df = pd.DataFrame({
    'title': X_title,
    'summary': X_summary,
    'true_topic': y_true,
    'predicted_topic': y_pred,
    'correct': [pred == true for pred, true in zip(y_pred, y_true)]
})

In [17]:
errors_df = results_df[~results_df['correct']]
errors_df.to_csv('classification_errors.csv', sep=';', index=False, encoding='utf-8-sig')